## Hexagonal Grid - How to create an equillateral hexagonal grid to be visualized in GIS

List of ingredients:

- Central points, containing horizontal cartesian coordinates (in X,Y format)
- Area of Interest / AOI (in this section we generate the shapefile first)
- Sidelength of Hexagon

In [87]:
# import library

import numpy as np
import matplotlib.pyplot as plt
import shapely.geometry as sg
import geopandas as gpd
from shapely.geometry import shape
from fiona import collection
from shapely.geometry import Point

### 1. Create a systematic central points function
---

in this part, we will try to generate central points at first, before try to reconstruct a polygon from the result

In [91]:
def generate_point_sequences(point_init, sidelength, cols):
    """Generates a sequence of points for a Cartesian planar diagram.

    Args:
        point_init: A list of x and y coordinates
        sidelength: The number of sidelength will be generated
        rows: the number of rows will be generated

    Returns:
        A list of shapely Point objects.
    """
    sequence_xy = []
    x_init = point_init[0] 
    y_init = point_init[1]
    
    for i in range(cols):
        x = i * 1.5 * sidelength
        y = i*np.sqrt(3)*1.5 * sidelength
        x_n = -x
        y_offset = []
        for a in range (cols - i):
            if (a + i) % 2 == 0:
                y_new = a * np.sqrt(3)/2 * sidelength
                y_offset.append(y_new)
        
        for idx,j in enumerate(y_offset):
            sequence_xy.append((x_init + x, y_init + j))
            sequence_xy.append((x_init - x, y_init - j))
            sequence_xy.append((x_init + x_n, y_init + j))
            sequence_xy.append((x_init - x_n, y_init - j))
            
    seq_pt = [Point(x, y) for (x, y) in sequence_xy] 
    return seq_pt

### 2. Create an equillateral hexagon shape function
---

In [93]:
def hex (sidelength, x_init, y_init):
    '''
    function to generate equillateral hexagon shape
    Parameter:
    - sidelength: (int/float) sidelength of the hexagon shape
    - x_init: (int/float) x coordinate of the hexagon to be initiated
    - y_init: (int/float) y coordinate of the hexagon to be initiated
    '''
    # initiate theta of equilateral hexagon, which is 120 degrees
    int_angle = 2*np.pi/3 

    #initiate an empty list
    coor_list = []

    # initiate first state of integer variable for looping purposes
    a = 0
    point_x = x_init
    point_y = y_init

    # do the loop for coordinate plotting
    while a < 6: 
        # state a first theta, and repeat by a += 1
        theta = a * int_angle
        a += 1

        # calculate x and y variable 
        x = sidelength*(np.cos(theta)) + point_x
        y = sidelength*(np.sin(theta)) + point_y
        
        
        # state a conditional statements on loop in even numbers
        if a % 2 == 0:
            x = sidelength*(np.cos(theta))*(-1) + point_x
            y = sidelength*(np.sin(theta))*(-1) + point_y
        
        # put all of the output to coor_list variable
        coor_list.append((x,y))

    # convert coor_list into array
    coord = np.array(coor_list)
        
    # calculate area and plot hexagon with coord variable
    area = (np.sqrt(3)/2)*(s**2)
    hexagon = sg.Polygon(coord)
    return hexagon

### 3. Create multiple grid from hexagon shape function
---

In [95]:
# grid generator function
def grid_hex(point_init, sidelength):
    '''
    function to generate hexagon shape grid
    Parameter:
    - s: (int) side length of the hexagon shape
    - point_init: (list) consist of x, y as an initial point placed at center

    Return:
    - hex_grid: (shape object) an output of shapely geometry
    '''
    hex_grid = []

    # initiate first state of integer variable for looping purposes
    point_x = point_init['X']
    point_y = point_init['Y']

    # do the loop for coordinate plotting
    for x, y in zip(point_x, point_y):
        hexagon = hex (sidelength, x_init = s, y_init = y)
        hex_grid.append(hexagon)

    return hex_grid

### 4. shapefile exporter function
---

In [97]:
def export_to_shapefile(target, filename, crs):
    """
    Converts a list of Shapely Point objects to a GeoDataFrame and exports it as a shapefile.
    
    Parameters:
        point_sequence (list): List of Shapely Point objects.
        filename (str): The output shapefile name (e.g., "points.shp").
        crs (int, optional): Coordinate Reference System (we will use 32748 for this occasion).
    """
    # Create a dictionary for the GeoDataFrame
    gdf = gpd.GeoDataFrame({"geometry": target})
    gdf = gdf.set_crs(crs)

    # export GeoDataFrame object into shapefile
    gdf = gdf.to_file(filename)
    
    return gdf

## Main code
---

In [163]:
# import shapefile and extract centroid from shape
AOI = gpd.read_file("AOI.shp")
AOI['centroid'] = AOI.geometry.centroid
x_values = AOI.geometry.centroid.x.tolist()
y_values = AOI.geometry.centroid.y.tolist()

In [165]:
point_init = [x_values[0], y_values[0]]
sidelength = 4000
cols = 25

In [167]:
# apply the function of generating sequences
sequences = generate_point_sequences(point_init, sidelength, cols)

In [169]:
# export to shapefile with shapefile function
crs = 32748
gdf = export_to_shapefile(target = sequences, filename = "hexagon.shp", crs)

In [171]:
points = gpd.read_file("hex_ctr_fin_2.shp")
points.info()
points['X']  = points.geometry.x
points['Y']  = points.geometry.y

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 676 entries, 0 to 675
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   FID       676 non-null    int64   
 1   geometry  676 non-null    geometry
dtypes: geometry(1), int64(1)
memory usage: 10.7 KB


In [173]:
hex_grid = grid_hex (point_init = points, sidelength = sidelength)

In [175]:
gdf_hex = export_to_shapefile(target = hex_grid, filename = "hex_fin_2.shp", crs=32748)